# Get dataset

In [27]:
import pandas as pd
import numpy as np
import random
import re  
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Templates

In [28]:
# templates
templates = pd.read_json("data/templates.json")
init_uttr_updated = pd.read_csv("data/InitUtterance_list_updated.csv")
templates_updated = templates.merge(init_uttr_updated, on='id')
templates_updated = templates_updated[['id','de', 'fr', 'it', 'en']]
templates_updated.head()

,id,de,fr,it,en
0,MyCloud.OpenArea.Init.Utterance,{'texts': [{'ttsText': '{MyCloudArea} auf #myC...,{'texts': [{'ttsText': 'afficher {MyCloudArea}...,{'texts': [{'ttsText': 'apri {MyCloudArea} su ...,{'texts': [{'ttsText': 'show {MyCloudArea} on ...
1,Router.SwitchOnWiFi.Init.Utterance,"{'texts': [{'ttsText': 'aktiviere das Wlan'}, ...","{'texts': [{'ttsText': 'active le Wi-Fi'}, {'t...","{'texts': [{'ttsText': 'attiva il #wifi'}, {'t...","{'texts': [{'ttsText': 'activate the Wi-Fi'}, ..."
2,Tv.GenericExitToTv.Init.Utterance,{'texts': [{'ttsText': 'gehe zurück zum Fernse...,"{'texts': [{'ttsText': 'revenir à la télé'}, {...","{'texts': [{'ttsText': 'torna alla TV'}, {'tts...","{'texts': [{'ttsText': 'back to TV'}, {'ttsTex..."
3,Router.SwitchOffWiFi.Init.Utterance,{'texts': [{'ttsText': 'deaktiviere das Wlan'}...,"{'texts': [{'ttsText': 'désactiver le Wi-Fi'},...","{'texts': [{'ttsText': 'disattiva il wifi'}, {...",{'texts': [{'ttsText': 'can you please switch ...
4,Tv.RadioChannelChange.Init.Utterance,{'texts': [{'ttsText': 'auf Radio {RadioChanne...,{'texts': [{'ttsText': 'prendre radio {RadioCh...,{'texts': [{'ttsText': 'metti radio {RadioChan...,{'texts': [{'ttsText': 'can you turn on radio ...


In [3]:
templates_updated.describe()

,id,de,fr,it,en
count,39,39,39,39,39
unique,39,39,39,39,39
top,Tv.VolumeDown.Init.Utterance,{'texts': [{'ttsText': '{SmartHomeSceneName} S...,{'texts': [{'ttsText': 'activer scène {SmartHo...,{'texts': [{'ttsText': 'attiva scena {SmartHom...,{'texts': [{'ttsText': 'activate {SmartHomeSce...
freq,1,1,1,1,1


## Entities

In [29]:
# entities
entities = pd.read_json("data/entities.json")
entities = entities[['value', 'type', 'language', 'normalizedValue', 'aliases']]
entities.head()

,value,type,language,normalizedValue,aliases
0,Widnau,LocalsearchLocation,de,widnau,[]
1,Zurigo,LocalsearchLocation,it,zurigo,[]
2,Solothurn,LocalsearchLocation,de,solothurn,[]
3,Muttenz,LocalsearchLocation,de,muttenz,[]
4,Lyss,LocalsearchLocation,de,lyss,[]


In [5]:
entities.describe()

,value,type,language,normalizedValue,aliases
count,65471,65471,65471,65471,65471
unique,51272,33,4,52082,3793
top,Golf,SeriesName,de,golf,[]
freq,20,17913,22746,20,59649


# Explore data

### Entities catagorical count

In [6]:
df = entities[['type', 'language']]
df = df.groupby(['type','language']).size().reset_index(name='count') 
df.pivot(index='type', columns='language', values='count')#.plot(kind='bar')

language,de,en,fr,it
type,,,,
AlbumName,NaN,1479.0,NaN,NaN
AppName,326.0,273.0,312.0,290.0
AreaName,26.0,26.0,26.0,26.0
ArtistName,NaN,822.0,NaN,NaN
BroadcastName,715.0,483.0,676.0,663.0
CallingNumberType,3.0,4.0,4.0,4.0
ClipName,353.0,320.0,342.0,338.0
DossierName,8.0,3.0,5.0,3.0
FirstName,952.0,NaN,NaN,NaN


### Generate correct command

In [129]:
from generator.tools import filter_aliases
l = [   "SRF 3",
		"D R S 3",
		"Schweiz 3",
		"S R F 3",
		"DRS 3",
        "s r f 3",
		"s r f drei",
		"d r s drei",
		"schweiz drei",
		"DRS drei",
        "one and a half man"
	]
l2 = [
		"D M A X",
		"d max",
		"DMAX",
		"d m a x",
		"D Max",
		"D Max Italia",
		"d max italia"
	]
l3 = [
		"TF three",
		"TF 3",
		"T F 3",
		"t f three"
	]
l4 = [
		"E V Z Academy",
		"e v z academy"
	]
l5 = [
		"BN",
		"b n t v",
		"B N",
		"b n",
		"BN TV",
		"B N T V"
	]
filter_aliases(l5,'de')

[&#39;BN&#39;, &#39;BNTV&#39;, &#39;BN TV&#39;]

In [130]:
l6 = [
		"S R F",
		"Schweiz 1",
		"SRF eis",
		"DRS 1",
		"SRF",
		"S R F eis",
		"s r f einz",
		"d r s eins",
		"S R",
		"schweizer",
		"Schweizer",
		"DRS eins",
		"SR",
		"SRF einz",
		"D R S",
		"s r f eis",
		"SRF 1",
		"D R S 1",
		"DRS",
		"s r",
		"S R F 1",
		"s r f eins",
		"SRF eins",
		"d r s",
		"S R F einz",
		"s r f",
		"schweiz eins"
	]
filter_aliases(l6, 'de')

[&#39;SRF eis&#39;,
 &#39;SR&#39;,
 &#39;Schweiz 1&#39;,
 &#39;DRS 1&#39;,
 &#39;SRF&#39;,
 &#39;Schweizer&#39;,
 &#39;DRS&#39;,
 &#39;SRF einz&#39;,
 &#39;SRF 1&#39;]

In [74]:
pd.concat([entities, a]).reset_index()

,index,value,type,language,normalizedValue,aliases
0,0,Widnau,LocalsearchLocation,de,widnau,[]
1,1,Zurigo,LocalsearchLocation,it,zurigo,[]
2,2,Solothurn,LocalsearchLocation,de,solothurn,[]
3,3,Muttenz,LocalsearchLocation,de,muttenz,[]
4,4,Lyss,LocalsearchLocation,de,lyss,[]
...,...,...,...,...,...,...
65866,395,98,RadioChannelPosition,it,novantotto,[]
65867,396,99,RadioChannelPosition,en,ninety nine,[]
65868,397,99,RadioChannelPosition,fr,quatre vingt dix neuf,[]
65869,398,99,RadioChannelPosition,de,neunundneunzig,[]


In [71]:
entities

,value,type,language,normalizedValue,aliases
0,Widnau,LocalsearchLocation,de,widnau,[]
1,Zurigo,LocalsearchLocation,it,zurigo,[]
2,Solothurn,LocalsearchLocation,de,solothurn,[]
3,Muttenz,LocalsearchLocation,de,muttenz,[]
4,Lyss,LocalsearchLocation,de,lyss,[]
...,...,...,...,...,...
65466,baby dance,SmartHomeSceneName,en,baby dance,[]
65467,Fernseher,SmartHomeSceneName,de,fernseher,[]
65468,tutto spento,SmartHomeSceneName,it,tutto spento,[]
65469,lumières,SmartHomeSceneName,fr,lumières,[]


In [134]:
from generator.module import Generator
from generator.normalizer import Normalizer
gen = Generator(templates=templates_updated, entities=pd.concat([entities, a]).reset_index(), method='all')
gen.get_command(target_id='Tv.RadioChannelChange.Init.Utterance', 
                target_lang='en', 
                verbose=False)[:250]

[&#39;can you turn on radio station 99&#39;,
 &#39;change radio station&#39;,
 &#39;change to radio channel 99 please&#39;,
 &#39;change the radio channel&#39;,
 &#39;change the radio station please&#39;,
 &#39;change the radio station to Freundes-Dienst Schweiz&#39;,
 &#39;change the radio station to Radio Freundes - Dienst&#39;,
 &#39;change the radio station to Radio Freundes-Dienst Schweiz&#39;,
 &#39;change the radio station to Radio Freundes-Dienst&#39;,
 &#39;change the radio station to Radio Freundes - Dienst Schweiz&#39;,
 &#39;change the radio station to Freundes - Dienst Schweiz&#39;,
 &#39;change the station please&#39;,
 &#39;could you please change the radio station to Freundes-Dienst Schweiz&#39;,
 &#39;could you please change the radio station to Radio Freundes - Dienst&#39;,
 &#39;could you please change the radio station to Radio Freundes-Dienst Schweiz&#39;,
 &#39;could you please change the radio station to Radio Freundes-Dienst&#39;,
 &#39;could you please change t

In [85]:
import random
k = entities[entities['type'] == 'MyCloudArea'][entities['language'] == 'de']
k

,value,type,language,normalizedValue,aliases
41769,Fotos,MyCloudArea,de,fotos,"[fotos, fotogalerie, Bilder, Bilder Galerie, m..."
41772,Favoriten,MyCloudArea,de,favoriten,"[Lieblingsfotos, Lieblingsfoto, lieblingsaufna..."
41774,Alben,MyCloudArea,de,alben,"[bilderalben, Foto Album, Fotoalben, Sammlung,..."
41776,Videos,MyCloudArea,de,videos,"[Video Aufnahmen, Videoaufnahmen, videoaufnahm..."
41780,Zuletzt hochgeladen,MyCloudArea,de,zuletzt hochgeladen,"[akutellen, letzte, aktuelle, jüngsten, neuest..."


In [136]:

gen.get_command(target_id='MyCloud.OpenArea.Init.Utterance', 
                    target_lang='de', 
                    verbose=False)

n&#39;,
 &#39;kann ich mir Bildersammlung bei myCloud anschauen&#39;,
 &#39;kann ich mir Alben bei myCloud anschauen&#39;,
 &#39;kann ich mir Fotoalbum bei myCloud anschauen&#39;,
 &#39;kann ich mir Foto Album bei myCloud anschauen&#39;,
 &#39;kann ich mir Fotoalben bei myCloud anschauen&#39;,
 &#39;kann ich mir Album bei myCloud anschauen&#39;,
 &#39;kann ich mir Bilderalben bei myCloud anschauen&#39;,
 &#39;kann ich mir Ordner bei myCloud anschauen&#39;,
 &#39;kann ich mir Video Aufnahmen bei myCloud anschauen&#39;,
 &#39;kann ich mir meine Filme bei myCloud anschauen&#39;,
 &#39;kann ich mir Videos bei myCloud anschauen&#39;,
 &#39;kann ich mir Videoaufnahmen bei myCloud anschauen&#39;,
 &#39;kann ich mir zuletzt hochgeladene bei myCloud anschauen&#39;,
 &#39;kann ich mir akutellen bei myCloud anschauen&#39;,
 &#39;kann ich mir jüngstes bei myCloud anschauen&#39;,
 &#39;kann ich mir neuesten bei myCloud anschauen&#39;,
 &#39;kann ich mir zuletzt hochgeladenes bei myCloud anschauen&#

In [17]:
normalizer = 
gen = Generator(templates=templates_updated, entities=entities)

gen.generate_command(target_id='Tv.GenericExitToTv.Init.Utterance', 
                    target_lang='de', 
                    normalizer= normalizer)

'wieder fernsehen'

In [54]:
import re

s

&#39;montrer film sur myCloud&#39;